In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

In [8]:
pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [ ]:
##  ------ api for langsmith, tavily tool(serach engine),  gemini

In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
os.environ["TAVILY_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

 ········
 ········
 ········


In [3]:
## creating tool

from langchain_tavily import TavilySearch

search = TavilySearch(
    name="weather_search",
    description="Use this tool to look up current weather in a specific location. Only use it for weather-related questions. Else answer by your own"
)
search_results = search.invoke("Who is gandhi")
print(search_results)


tools = [search]

{'query': 'Who is gandhi', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://en.wikipedia.org/wiki/Mahatma_Gandhi', 'title': 'Mahatma Gandhi - Wikipedia', 'content': "**Mohandas Karamchand Gandhi** (2 October 1869– 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule.", 'score': 0.8684817, 'raw_content': None}, {'url': 'https://www.britannica.com/biography/Mahatma-Gandhi', 'title': 'Mahatma Gandhi | Biography, Education, Religion ... - Britannica', 'content': 'Mahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, & Facts | Britannica Mahatma Gandhi Mahatma Gandhi Timeline Gandhi and Indian History Mahatma Gandhi (more) Mahatma Gandhi His family stayed in India while Gandhi went to London in 1888 to study law and to South Africa in 1893 to practice it. Mahatma Gandhi (born October 2, 186

In [4]:
# llm

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [5]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

In [6]:
##------------ binding model with tools-------------------------##

model_with_tools = model.bind_tools(tools)

In [7]:
##------------ model with tool and there was no tool call --------------------------##


query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi there! How can I help you today?

Tool calls: []


In [8]:
##------------ model with tool and there was tool call --------------------------##

user_msg="what is the weather in chennai"
query=f"{user_msg}"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'weather_search', 'args': {'query': 'weather in chennai'}, 'id': '33ce1855-505a-4ebe-a8a9-ec7da10b9959', 'type': 'tool_call'}]


In [10]:
##-------------- creating agent---------------##

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [12]:
#--------invoking the agent and no tool call---------------##
query="Hii"
input_message = {"role": "user", "content":query}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hii
================================== Ai Message ==================================

Hello! How can I help you today?


In [14]:
##-----------------invoking the agent with tool call----------------------##

input_message = {"role": "user", "content": "weather in paris"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

weather in paris
================================== Ai Message ==================================
Tool Calls:
  weather_search (4b38aee5-1ed9-480b-93c9-045ab9e16cc4)
 Call ID: 4b38aee5-1ed9-480b-93c9-045ab9e16cc4
  Args:
    query: weather in paris
================================= Tool Message =================================
Name: weather_search

{"query": "weather in paris", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Paris", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.8667, 'lon': 2.3333, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1751877803, 'localtime': '2025-07-07 10:43'}, 'current': {'last_updated_epoch': 1751877000, 'last_updated': '2025-07-07 10:30', 'temp_c': 15.3, 'temp_f': 59.5, 'is_day': 1, 'condition': {'text': 'Moderate or heavy rain with thunder

In [15]:
## stream messages
input_message = {"role": "user", "content": "weather in Chennai, India"}
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

weather in Chennai, India
================================== Ai Message ==================================
Tool Calls:
  weather_search (5ef3bb95-09f0-4d68-a628-27f23fb6c2c1)
 Call ID: 5ef3bb95-09f0-4d68-a628-27f23fb6c2c1
  Args:
    query: weather in Chennai, India
================================= Tool Message =================================
Name: weather_search

{"query": "weather in Chennai, India", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Chennai, India", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.0833, 'lon': 80.2833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1751879876, 'localtime': '2025-07-07 14:47'}, 'current': {'last_updated_epoch': 1751879700, 'last_updated': '2025-07-07 14:45', 'temp_c': 37.3, 'temp_f': 99.1, 'is_day': 1, 'condition': {'text': '

In [16]:
## stream tokens

for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in Chennai|, India is partly cloudy with a temperature of 37.3°C (99|.1°F). It feels like 41.5°C (10|6.7°F). The wind is blowing from the west at 23.8 km/h. The humidity is 48%.|

In [17]:
##------------------making the agent which is stateless to keep memory of previous conversations-------------------##

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()



In [18]:
##----------- now along with model, tool we are passing memory to the agent-----------------##


agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "xyc123"}}

In [19]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hi Bob! How can I help you today?


In [20]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob.


In [21]:
input_message = {"role": "user", "content": "weather in coimbatore"}
i=1
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    if(i==1):
      step["messages"][-1].pretty_print()
    ans=step["messages"][-1]
    i+=1
ans.pretty_print()

================================ Human Message =================================

weather in coimbatore
================================== Ai Message ==================================

The weather in Coimbatore, India is partly cloudy with a temperature of 32.1°C (89.8°F). The wind is from the WSW at 31.0 kph. The humidity is 56%.
